# Деревья решений 
В этом задании предстоит реализовать очень полезный метод классификации - дерево решений. 

Одной из его особенностей является возможность объяснить в человекочитаемой форме, почему мы отнесли объект к определенному классу. Эта особенность позволяет использовать деревья решений для создания систем, которые могут подсказывать специалистам, на что именно стоит обратить внимание при принятии решений. Кроме того, этот метод часто используется в случаях, когда интерпретируемость модели имеет большое значение, например, в чувствительных задачах 

In [1]:
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib

Основная идея любого алгоритма дерева решений заключается в следующем: 
1. Выберите лучший атрибут, используя меры выбора атрибута (ASM), чтобы разделить примеры. 
2.  Сделайте этот атрибут узлом решения и разбейте набор данных на более мелкие подмножества. 
3. Начните построение дерева, рекурсивно повторяя этот процесс для каждого дочернего элемента, пока не совпадет одно из условий:
   1. Все кортежи принадлежат одному и тому же значению атрибута. 
   2. Оставшихся атрибутов больше нет. 
   3. Больше нет примеров

## Использование различных моделей

Протестируем решение на датасетах [mushrooms](https://www.kaggle.com/datasets/uciml/mushroom-classification) и diabetes?.
1. Выполним загрузку и предобработку данных.
2. Разобьем данные на тренировочный и валидационный набор для оценки точности работы алгоритма.
3. Посчитаем метрики для различных параметров построения дерева

In [1]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
import yaml
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
with open('../config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

Начнем с WDBC. В этом датасете хранятся геометрические и прочие внешние признаки опухолей (вероятно, полученные из МРТ). Опухоли будут быть доброкачественными (benign = B) и злокачественными (malignant = M). Задача состоит в определении типа опухоли по данным признакам

In [6]:
df = pd.read_csv(cfg["classification"]["wdbc"])
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


Как обычно, отбрасываем идентификатор

In [7]:
df = df.drop(['id', 'Unnamed: 32'], axis=1)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Трансформируем строковые категории B и M в числовые 0 и 1, после чего разделяем признаки и таргет, который будем предсказывать. Это дает нам датафреймы X и y соответственно.

In [ ]:
df['diagnosis'] = df['diagnosis'].replace({'B': 0, 'M': 1}).astype(int)
target = 'diagnosis'
features = list(df.columns)
features.remove('diagnosis')
features

In [ ]:
X_diagnosis = df[features]
y_diagnosis = df[[target]]

Рассмотрим теперь датасет mushrooms. В нем содержатся геометрические и прочие внешние признаки грибов, которые классифицируются на съедобные и ядовитые. Наша задача состоит в подобной классификации по данным признакам

In [ ]:
df = pd.read_csv(cfg["classification"]["mushrooms"])
target = 'class'
features = list(df.columns)
features.remove(target)
X_mushroom = df[features]
y_mushroom = df[[target]]
X_mushroom.head()

**Задание**: Проведите краткий EDA. Есть ли выбросы в данных, как связаны столбцы? Хватит 2-3 графиков или таблиц (но можно больше). Какие есть типы признаков в этом датасете?

In [ ]:
X_mushroom.info()

In [ ]:
import category_encoders as ce

encoder = ce.CountEncoder()
X_mushroom = encoder.fit_transform(X_mushroom)
X_mushroom.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_mushroom = encoder.fit_transform(y_mushroom)

Теперь перейдем к предсказанию типа опухолей с помощью деревьев решений.

Проведем train/test сплит

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_diagnosis, y_diagnosis, test_size=0.2, random_state=1)

In [ ]:
y_train

Поэкспериментируем с деревом решений из библиотеки sklearn. 

In [ ]:
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=30)
tree.fit(X_train, y_train)

In [ ]:
tree.predict_proba(X_test)[:, 1]

In [ ]:
tree.score(X_test, y_test)
#metrics.roc_auc_score(y_test, tree.predict_proba(X_test)[:, 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
plot_tree(tree, ax=ax)

Рассмотрим другие значения гиперпараметров

In [ ]:
tree_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
plot_tree(tree_gini.fit(X_train, y_train), ax=ax)

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(tree_gini.score(X_train, y_train)))
print('Test-set accuracy score: {0:0.4f}'. format(tree_gini.score(X_test, y_test)))

Попробуем вместо Джини использовать энтропию

In [ ]:
# Постройте дерево решений с использованием энтропии.
################
# YOUR CODE HERE
################

In [ ]:
print('Training set score: {:.4f}'.format(tree_en.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(tree_en.score(X_test, y_test)))

Перейдем теперь к датасету с грибами

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_mushroom, y_mushroom, test_size=0.2, random_state=1)
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=30)
plot_tree(tree.fit(X_train, y_train))

In [ ]:
y_pred_train = tree.predict(X_train)
y_pred_test = tree.predict(X_test)
print('Training-set accuracy score: {0:0.4f}'. format(tree.score(X_train, y_train)))
print('Test-set accuracy score: {0:0.4f}'. format(tree.score(X_test, y_test)))

Сравним, как разные классификаторы строят границы разделения. Для начала посмотрим на общую картинку, как в зависимости от настроек будет отличаться дерево.

In [ ]:
classifiers = {
    "DT_basic_3": DecisionTreeClassifier(max_depth=3),
    "DT_basic_5_no_min": DecisionTreeClassifier(max_depth=5),
    "DT_basic_5": DecisionTreeClassifier(max_depth=5, min_samples_leaf=30),
    "DT_gini_3": DecisionTreeClassifier(max_depth=3, criterion='gini',  min_samples_leaf=30),
    "DT_gini_5": DecisionTreeClassifier(max_depth=5, criterion='gini', min_samples_leaf=30),
    "DT_entropy_3": DecisionTreeClassifier(max_depth=3, criterion='entropy', min_samples_leaf=30),
    "DT_entropy_5": DecisionTreeClassifier(max_depth=5, criterion='entropy', min_samples_leaf=30),
    "DT_entropy_5_no_min": DecisionTreeClassifier(max_depth=5, criterion='entropy'),
}
    

columns =  X_diagnosis.columns[:2] 
columns_mushroom = ['gill-color', 'cap-color']

datasets = [
    (X_diagnosis[columns].to_numpy(), y_diagnosis.to_numpy()),
    (X_mushroom[columns_mushroom].to_numpy(), y_mushroom),
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1)
]


In [ ]:
def set_grid(ax, i, j, x_min, x_max, y_min, y_max):
    ax[i][j].set_xlim(x_min, x_max)
    ax[i][j].set_ylim(y_min, y_max)
    ax[i][j].set_xticks(())
    ax[i][j].set_yticks(())

In [ ]:
fig, ax = plt.subplots(len(datasets), len(classifiers)+1, figsize=(15, 10))

for dataset_num, data in enumerate(datasets):
    X, y = data
    X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.3, random_state=42
      )
    diff_x = X_train[:, 0].max() - X_train[:, 0].min()
    diff_y = X_train[:, 1].max() - X_train[:, 1].min()
    x_min, x_max = X_train[:, 0].min() - diff_x*0.1, X_train[:, 0].max() + diff_x*0.1
    y_min, y_max = X_train[:, 1].min() - diff_y*0.1, X_train[:, 1].max() + diff_y*0.1


    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])
    if dataset_num == 0:
          ax[dataset_num][0].set_title("Input data")
            
    # Plot the training points
    ax[dataset_num][0].scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k")
    # Plot the testing points
    ax[dataset_num][0].scatter(
      X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6, edgecolors="k"
    )
    set_grid(ax, dataset_num, 0, x_min, x_max, y_min, y_max)
    
    # iterate over classifiers
    for cls_num, (name, clf) in enumerate(classifiers.items(), start=1):
        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        DecisionBoundaryDisplay.from_estimator(
            clf, X_train, cmap=cm, alpha=0.8, ax=ax[dataset_num][cls_num], eps=0.5
        )
        
        # Plot the training points
        #ax[dataset_num][cls_num].scatter(
        #    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
        #)
        
        # Plot the testing points
        ax[dataset_num][cls_num].scatter(
            X_test[:, 0],
            X_test[:, 1],
            c=y_test,
            cmap=cm_bright,
            edgecolors="k",
            alpha=0.6,
        )

        set_grid(ax, dataset_num, cls_num, x_min, x_max, y_min, y_max)
        if dataset_num == 0:
            ax[dataset_num][cls_num].set_title(name, fontdict={'fontsize': 10, 'fontweight': 'medium'})
            
        ax[dataset_num][cls_num].text(
            x_max - 0.3,
            y_min + 0.3,
            ("%.2f" % score).lstrip("0"),
            size=15,
            horizontalalignment="right",
        )

plt.tight_layout()
plt.show()

Деревья решений легко переобучаются. В теории, каждая точка набора может сформировать листовую вершину. Поэтому всегда надо аккуратно выбирать гиперпараметры, влияющие на разбиение дерева. Кроме того, деревья чувствительны к обучающе выборке. Даже небольшая ее пертурбация может привести к очень серьезным изменениям в классификаторе.

**Задание**: 
1) Постройте несколько графиков, чтобы оценить, как будет выглядить разделение плоскости в зависимости от 
    - минимального количества объектов в листе
    - максимальной глубины дерева
  К увеличению или уменьшению качества на обучающей выборке приводит увеличение глубины дерева? А на тестовой? 
2) Постройте несколько графиков, чтобы оценить, как будет выглядить разделение плоскости в зависимости от подвыборки. Выберите из вашего обучающего набора 90% семплов с разными сидами и посмотрите, как поменяются предсказания.

**Задание:**
  Для датасета mushrooms сравните, как меняется точность и переобучение для деревьев с разными кодировками признаков. Можете зафиксировать остальные параметры. 